# Gini 네비게이션의 RecentData.bin 파일 분석

+ Gini 네비게이션에서 특정 위치를 설정하거나 검색 기능을 사용하여 특정 위치를 출발지 또는 도착지로 설정한 경우 지점의 정보를 저장.
+ RecentData.bin 파일은 UserData에 존재.

In [1]:
import struct
import time
import folium

In [116]:
Last_Data = open('C:/Users/LEEKEONYONG/Desktop/navigation_backup/UserData/RecentData.bin', 'rb')
data = Last_Data.read()
Last_Data.close()

### 메소드 설명
 + Gini 네비게이션 상의 RecentData.bin 파일의 데이터 분석을 위한 함수 정의
 + 

In [122]:
def GiniRecentDatabin(data):
    ls_Search_Data = []
    
    Offset_in_GiniRecentDatabin = 110
    Data_Except_Removed = data[8]
    All_Data = data[11]
    
    for i in range(0,110*All_Data,Offset_in_GiniRecentDatabin):
        
        Data_Seq = data[12+i]
        
        Search_Time = struct.unpack('<HH',data[15+i:15+4+i])
        Search_Time = (Search_Time[1]<<16)+Search_Time[0]
        Search_Time = time.gmtime(Search_Time)
        Search_Time = str(Search_Time[0])+'년 '+str(Search_Time[1])+'월 '+str(Search_Time[2])+'일 '+str(Search_Time[3])+'시 '+str(Search_Time[4])+'분'
        
        Data_Classfication = data[44+i]
        if Data_Classfication == 0:
            Data_Classfication = '최근검색지'
        elif Data_Classfication == 1:
            Data_Classfication = '최근출발지'
        elif Data_Classfication == 2:
            Data_Classfication = '최근목적지'
        elif Data_Classfication == 3:
            Data_Classfication = '최근경유지'
            
        X_Line = data[29+i:29+i+4]
        Y_Line = data[29+i+4:29+i+8]

        X_Line = struct.unpack('<HH',X_Line)
        X_Line = (X_Line[1]<<16)+X_Line[0]

        Y_Line = struct.unpack('<HH',Y_Line)
        Y_Line = (Y_Line[1]<<16)+Y_Line[0]
        
#         X_Line += 81284000
#         Y_Line += 23820000
        
        Search_Place = data[51+i:51+i+data[51+i-1]*2].decode('utf-16')
        
        ls_Search_Data.append((Data_Seq, Search_Time, Data_Classfication, (Y_Line,X_Line), Search_Place))
        
    return [Data_Except_Removed,All_Data,ls_Search_Data]

In [123]:
GiniRecentData = GiniRecentDatabin(data)

In [124]:
print('삭제된 데이터를 제외한 개수: '+str(GiniRecentData[0]))

삭제된 데이터를 제외한 개수: 20


In [125]:
print('총 데이터의 개수: ' + str(GiniRecentData[1]))

총 데이터의 개수: 20


In [126]:
for i in range(GiniRecentData[1]):
    print(GiniRecentData[2][i])

(1, '2019년 7월 2일 14시 2분', '최근검색지', (13314600, 46059730), '충청북도 충주시 금곡서2길 19-1')
(2, '2019년 5월 28일 5시 7분', '최근검색지', (13314309, 46052317), '충북 충주시 칠금.금릉동')
(3, '2019년 7월 2일 14시 2분', '최근검색지', (13314600, 46059730), '충청북도 충주시 금곡서2길 19-1')
(4, '2019년 5월 29일 5시 29분', '최근검색지', (13313174, 46059030), '충북 충주시 연수동')
(5, '2019년 6월 3일 12시 17분', '최근검색지', (13242618, 45931721), '충북 증평군 증평읍')
(6, '2019년 6월 10일 11시 37분', '최근검색지', (13187612, 45892284), '충북 청주시서원구 사직2동')
(7, '2019년 7월 2일 14시 2분', '최근검색지', (13314351, 46056913), '충북 충주시 연수동')
(8, '2019년 8월 9일 12시 42분', '최근검색지', (13312120, 46050188), '충주터미널')
(9, '2019년 12월 19일 12시 46분', '최근검색지', (13314394, 46059376), '충북 충주시 연수동')
(10, '2019년 8월 9일 12시 42분', '최근검색지', (13312120, 46050188), '충주터미널')
(11, '2019년 12월 19일 12시 46분', '최근검색지', (13312120, 46050188), '충주고속터미널')
(12, '2019년 12월 19일 12시 46분', '최근검색지', (13312120, 46050188), '충주고속터미널')
(13, '2019년 12월 25일 12시 59분', '최근검색지', (13308469, 46051097), '삼성디지털플라자(충주문화점)')
(14, '2019년 12월 25일 12시 59분', '최근검색지', (1

In [71]:
m = folium.Map([GPS[0]/10**6,GPS[1]/10**6], zoom_start = 15)
folium.Marker([GPS[0]/10**6,GPS[1]/10**6],tooltip = '해당지점 좌표').add_to(m)
m